In [1]:
using NBInclude
@nbinclude("dcj_algo.ipynb")

In [2]:
# id_counter = Ref{Int}(1)
# id_to_char = Dict{Int, Char}()
# char_to_id = Dict{Char, Int}()

# s = ".a."

# genome = string_to_genome(s, id_counter, id_to_char, char_to_id, true)

# # show(genome)

# adj_list, adj_set = genome_to_adj_listset(genome)   

# show(adj_list)

In [11]:
# 2 DCJ op - adj --> adj  
src = ".abc."
target = ".acb."

# 1 DCJ op - adj --> telo  
src = "abca"
target = ".abc."

# 1 DCJ op - telo --> adj  
src = ".ab.cc"
target = ".abc."



src = ".abcdefg."
target = ".abcdGFE." 

calculate_distance(src, target)

SRC ADJ LIST
(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,f:t)(f:h,g:t)(g:h,.)
|
v
TARGET ADJ LIST
(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,g:h)(g:t,f:h)(f:t,e:h)(e:t,.)

*************
indexing through target adj list...

(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,f:t)(f:h,g:t)(g:h,

.)
ADJ 2 :::: (a:h,b:t)


(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,f:t)(f:h,g:t)(g:h,.)
ADJ 3 :::: (b:h,c:t)


(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,f:t)(f:h,g:t)(g:h,.)
ADJ 4 :::: (c:h,d:t)


(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,f:t)(f:h,g:t)(g:h,.)
ADJ 5 :::: (d:h,g:h)
prev adj list ::: (.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,f:t)(f:h,g:t)(g:h,.)
updated adj list ::: (.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,g:h)(f:h,g:t)(e:h,f:t)(e:t,.)

(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,g:h)(f:h,g:t)(e:h,f:t)(e:t,.)
ADJ 6 :::: (g:t,f:h)


(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,g:h)(f:h,g:t)(e:h,f:t)(e:t,.)
ADJ 7 :::: (f:t,e:h)


(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,g:h)(f:h,g:t)(e:h,f:t)(e:t,.)
ADJ 1 :::: (.,a:t)


(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,g:h)(f:h,g:t)(e:h,f:t)(e:t,.)
ADJ 8 :::: (e:t,.)


1